## Harrison Durbin Technical Test - TomTom - 30.10.2016

In [1]:
import shapefile
import numpy as np
from scipy.spatial.distance import cdist
import time
import difflib as dl

In [2]:
gap_tolerance = 300 # meters

In [3]:
name_tolerance = 90 

In [4]:
sf = shapefile.Reader("NW_test/NW_test_cleaned")

In [5]:
shapes = sf.shapes()

In [6]:
fields = sf.fields
fields

[('DeletionFlag', 'C', 1, 0), ['LEFTNAME', 'C', 254, 0]]

In [7]:
records = sf.records()

In [8]:
records_unique = np.unique(records)
len(records_unique)

850

In [9]:
records[0]

['Avenida Marcelino Pires']

In [10]:
bbox = np.zeros(shape=(len(shapes),4))
for i in range(len(shapes)):
    bbox[i] = shapes[i].bbox

In [11]:
coords = np.zeros(shape=(len(records),4)) # coords in meters of bounding box
                     
for i in range(len(records)):
    coords[i,0] = int((-bbox[i][0] - 54)*103262) # lng
    coords[i,1] = int((-bbox[i][1] - 22)*110730) # lat
    coords[i,2] = int((-bbox[i][2] - 54)*103262) # lng
    coords[i,3] = int((-bbox[i][3] - 22)*110730) # lat

In [12]:
np.shape(coords)

(9001L, 4L)

In [13]:
coords[0]

array([ 77063.,  24567.,  77013.,  24563.])

In [14]:
records[0]

['Avenida Marcelino Pires']

In [15]:
id = np.arange(len(records))

In [16]:
raw_dict = dict(zip(id, records[:]))

In [17]:
dist_matrix = np.zeros(shape=(len(shapes),len(shapes)))
dist_matrix[:,:] = 1e8

In [38]:
# name_match_matrix = np.zeros(shape=(len(shapes),len(shapes)))
# for i in range(len(records)):
#     if i % 100 == 0:
#         print i
#     for j in range(len(records)):
#         s = 100*int((dl.SequenceMatcher(None, records[i], records[j])).quick_ratio())
#         name_match_matrix[i,j]=s

In [37]:
# np.savetxt("name_match_matrix.csv", name_match_matrix, fmt='%.3f', delimiter=",")

In [ ]:
name_match_matrix = np.genfromtxt('name_match_matrix.csv', delimiter=',')

In [23]:
# filling in distance matrix (distances from top right to lower left of different bounding boxes)
ti=time.time()
for i in range(len(records)):
#     if i % 100 == 0:
#         print i
    for j in range(len(records)):
        if records[i] != records[j]:
            s = name_match_matrix[i,j]
#         if records[i]==records[j] and i!=j:
            if s > name_tolerance and i!=j:            
                dist_matrix[i,j]=abs(int(cdist([coords[i][0:2]],[coords[j][2:4]])[0]))
            
print 'Run-time was' , int(time.time()-ti), 'seconds.'

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220


KeyboardInterrupt: 

In [ ]:
connections1 =[]
connections2 =[]

ti=time.time()

for j in range(len(records)):
    s = (dl.SequenceMatcher(None, raw_dict[dist_matrix[:,j].argmin()], records[j])).quick_ratio()
    if min(dist_matrix[:,j])<gap_tolerance and s > name_tolerance: 
#     if min(dist_matrix[:,j])<gap_tolerance and raw_dict[dist_matrix[:,j].argmin()]==records[j]: 
        connections1.append(dist_matrix[:,j].argmin())
    else:
        connections1.append(-1)

for i in range(len(records)):
    if i in connections1:    
        x = connections1.index(i)
        connections2.append(x)
    else:
        connections2.append(-1)
        
print 'Run-time was' , int(time.time()-ti), 'seconds.'

In [ ]:
isolated = []
part_of_chain = []
for i in range(len(records)):
    if connections1[i]==-1 and connections2[i]==-1:
        isolated.append(i)
    elif connections1[i]!=-1 and connections2[i]!=-1:
        part_of_chain.append(i)

In [ ]:
road_id = np.zeros(shape=(len(shapes),1))

In [ ]:
ti=time.time()

road_id = np.zeros(shape=(len(shapes),1))
counter = 1
traversed = []


for nodei in part_of_chain:

    if nodei in part_of_chain:
        chain = []
        node = nodei
        chain.append(node)
        
        while node != -1:
            node = connections1[node]
            if node != -1 and node not in chain:
                chain.append(node)          

            else:
                break
        
        node = nodei
        while node != -1:
            node = connections2[node]
            if node != -1  and node not in chain:
                chain.append(node)
            else:
                break

        for i in chain:
            if road_id[i]==0 and i not in traversed:
                traversed.append(i)
                road_id[i] = counter     
        counter+=1
        
        chain.sort()
#         print chain

print 'Run-time was' , int(time.time()-ti), 'seconds.'


In [ ]:
unique_ids = np.unique(road_id)

counter = 1
for i in range(len(records)):
    while counter in unique_ids:
        counter+=1
    if road_id[i]==0:
        road_id[i]=counter
        counter+=1

In [ ]:

# Read in our existing shapefile
r = sf

# Create a new shapefile in memory
w = shapefile.Writer()

# Copy over the existing fields
w.fields = list(r.fields)

# Add our new field using the pyshp API
w.field("road_id", "C", "40")

x=0
# Loop through each record, add a column.
for rec in r.records():
    rec.append(road_id[x])
    x+=1
    # Add the modified record to the new shapefile 
    w.records.append(rec)

# Copy over the geometry without any changes
w._shapes.extend(r.shapes())

# Save as a new shapefile (or write over the old one)
w.save("Harrison_Durbin_GISENG_Test_Solution") 